In [3]:
# !pip3 install --break-system-packages requests obspy
# !pip3 install --break-system-packages requests pyzt

In [ ]:
import requests
from datetime import datetime, timedelta, timezone
from obspy import read
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from scipy.io import wavfile
import matplotlib.dates as mdates


# Function to compute time window based on number of days
def compute_time_window(days):
    # Manually calculate the Alaska timezone offset
    utc_offset = timedelta(hours=-8)  # Alaska is UTC-9
    end_time = datetime.now(timezone.utc) + utc_offset
    start_time = end_time - timedelta(days=days)
    end_str = end_time.strftime("%Y-%m-%dT%H:%M:%S")
    start_str = start_time.strftime("%Y-%m-%dT%H:%M:%S")
    return start_str, end_str, end_time

# Specify the number of days
days = 1  # Change this value to the desired number of days

# Step 1: Compute time window
start_str, end_str, end_time = compute_time_window(days)

# Create directories if they don't exist
mseed_dir = "mseed_files"
audio_marker_dir = "Audio_Files"
plot_dir = "Plot_Files"
os.makedirs(mseed_dir, exist_ok=True)
os.makedirs(audio_marker_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

# Create filenames with the new format
formatted_time = end_time.strftime('%Y-%m-%d_T_%H%M%S')
filename = os.path.join(mseed_dir, f"Spurr_Last_{days}d_{formatted_time}.mseed")
audio_filename = os.path.join(audio_marker_dir, f"Spurr_Last_{days}d_{formatted_time}.wav")
marker_filename = os.path.join(audio_marker_dir, f"Spurr_Last_{days}d_{formatted_time}_Marker_File.txt")
plot_filename = os.path.join(plot_dir, f"Spurr_Last_{days}d_{formatted_time}.png")

# Step 2: Check if file exists locally or fetch MiniSEED data
if os.path.exists(filename):
    print(f"✅ Using existing file: {filename}")
    st = read(filename)
else:
    url = "https://service.iris.edu/fdsnws/dataselect/1/query"
    params = {
        "net": "AV",
        "sta": "SPCN",
        "loc": "--",
        "cha": "BHZ",
        "start": start_str,
        "end": end_str,
        "format": "miniseed",
        "nodata": 404
    }
    response = requests.get(url, params=params)

    # Step 3: Save if successful
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"✅ Downloaded and saved file: {filename}")
        st = read(filename)
    else:
        print(f"❌ Error {response.status_code}: No data found or request failed.")
        exit()

# Step 4: Plot waveform - Direct matplotlib approach
print(f"Stream contains {len(st)} traces")
print(f"Data points in first trace: {len(st[0].data)}")
print(f"Min/Max values: {st[0].data.min()}, {st[0].data.max()}")

# Define the tick interval (this was missing)
tick_interval_hours = 1

# Set global font size configurations
mpl.rcParams.update({
    'font.size': 12,           # Base font size
    'axes.titlesize': 16,      # Title font size
    'axes.labelsize': 14,      # Axis label font size
    'xtick.labelsize': 12,     # X-axis tick label size
    'ytick.labelsize': 12,     # Y-axis tick label size
    'legend.fontsize': 12,     # Legend font size
    'figure.titlesize': 18     # Figure title size
})


# Create a direct matplotlib plot
plt.figure(figsize=(16, 5))

# Get the time array for x-axis
tr = st[0]
times = tr.times("matplotlib")  # Convert to matplotlib date format
amplitude = tr.data

# Plot the data directly
plt.plot(times, amplitude)

# Format x-axis
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=tick_interval_hours))

# Format the end time for the title
formatted_end_time = end_time.strftime('%b %d %H:%M')

# Add labels and title with date information
plt.title(f"Spurr Seismic Data (SPCN) - Last {days} day(s) before {formatted_end_time}")
plt.ylabel("Amplitude")
plt.grid(True, alpha=0.3)

# Make sure everything fits
plt.tight_layout()

# Save and show
plt.savefig(plot_filename)
plt.show()
print(f"✅ Saved plot file: {plot_filename}")

# Step 5: Create audio file
# First normalize and prepare the data
tr = st[0].copy()
tr.detrend('demean')  # Remove mean
tr.taper(max_percentage=0.0001)  # Apply taper to reduce edge effects

# Set the sampling rate to 44100 Hz
tr.stats.sampling_rate = 882000

# Normalize amplitude to range [-1, 1] for audio
data = tr.data.astype(float)
max_amp = max(abs(data))
if max_amp > 0:  # Avoid division by zero
    data = data / max_amp

# Convert to 16-bit PCM
audio_data = (data * 32767).astype('int16')

# Save as WAV file
wavfile.write(audio_filename, int(tr.stats.sampling_rate), audio_data)
print(f"✅ Saved audio file: {audio_filename}")

# Function to generate marker file with timestamps aligned to day boundaries
def generate_marker_file_with_data_timestamps(stream, marker_interval_hours, filename, use_am_pm=True):
    with open(filename, "w") as f:
        f.write("Marker file version: 1\n")
        f.write("Time format: Samples\n")
        
        # Extract the start and end times from the stream
        # Convert ObsPy UTCDateTime to Python datetime
        obspy_start_time = stream[0].stats.starttime
        obspy_end_time = stream[0].stats.endtime
        
        # Convert to standard Python datetime objects
        start_time = datetime(
            obspy_start_time.year, obspy_start_time.month, obspy_start_time.day,
            obspy_start_time.hour, obspy_start_time.minute, obspy_start_time.second,
            obspy_start_time.microsecond
        )
        
        end_time = datetime(
            obspy_end_time.year, obspy_end_time.month, obspy_end_time.day,
            obspy_end_time.hour, obspy_end_time.minute, obspy_end_time.second,
            obspy_end_time.microsecond
        )
        
        sampling_rate = stream[0].stats.sampling_rate
        
        # Find the "anchor points" that align with day boundaries
        # For 6-hour intervals, these would be 00:00, 06:00, 12:00, 18:00
        
        # First, get the most recent anchor time before the start time
        anchor_time = start_time.replace(hour=0, minute=0, second=0, microsecond=0)
        
        # Now find the first anchor time that occurs at or after start_time
        current_time = anchor_time
        while current_time < start_time:
            current_time += timedelta(hours=marker_interval_hours)
            # If we pass into a new day, make sure we reset to hour intervals from midnight
            if current_time.hour < (current_time - timedelta(hours=marker_interval_hours)).hour:
                # We crossed midnight - align to midnight + interval pattern
                current_time = current_time.replace(hour=0, minute=0, second=0, microsecond=0)
                # Then advance by intervals until we're beyond start_time
                while current_time < start_time:
                    current_time += timedelta(hours=marker_interval_hours)
        
        # Now we have the first valid marker time after start_time
        # Write markers at the specified interval
        while current_time <= end_time:
            hour = current_time.hour
            if use_am_pm:
                formatted_time = current_time.strftime(f'%m/%d %I:%M %p')
            else:
                formatted_time = current_time.strftime(f'%m/%d %H:%M')
            
            # Calculate sample position
            seconds_from_start = (current_time - start_time).total_seconds()
            current_sample = int(seconds_from_start * sampling_rate)
            
            # Only write the marker if it falls within the data range
            if 0 <= current_sample <= len(stream[0].data):
                f.write(f"{formatted_time}\t{current_sample}\n")
            
            # Move to next marker time
            current_time += timedelta(hours=marker_interval_hours)
            
            # If we cross over to a new day, realign to the proper hour pattern
            if current_time.hour < (current_time - timedelta(hours=marker_interval_hours)).hour:
                # We crossed midnight - align to midnight + interval pattern
                current_time = current_time.replace(hour=0, minute=0, second=0, microsecond=0)
        
        # Write the end marker if it doesn't align with the interval markers
        end_hour = end_time.hour
        if use_am_pm:
            formatted_end_time = end_time.strftime(f'%m/%d %I:%M %p')
        else:
            formatted_end_time = end_time.strftime(f'%m/%d %H:%M')
        end_sample = len(stream[0].data) - 1
        
        # Only write if the last interval marker isn't already at the end
        last_marker_seconds = (current_time - timedelta(hours=marker_interval_hours) - end_time).total_seconds()
        if abs(last_marker_seconds) > 60:
            f.write(f"{formatted_end_time}\t{end_sample}\n")

marker_interval_hours = 2  # Set the interval for markers in hours

generate_marker_file_with_data_timestamps(st, marker_interval_hours, marker_filename, use_am_pm=True)
print(f"✅ Generated marker file: {marker_filename}")
